# Load data file

In [34]:
using DelimitedFiles, Test, BenchmarkTools, Statistics

"""General Annealing Problem"""
abstract type AnnealingProblem end

"""
    SpinAnnealingProblem{T<:Real} <: AnnealingProblem

Annealing problem defined by coupling matrix of spins.
"""
struct SpinAnnealingProblem{T<:Real} <: AnnealingProblem  # immutable, with type parameter T (a subtype of Real).
    num_spin::Int
    coupling::Matrix{T}
    function SpinAnnealingProblem(coupling::Matrix{T}) where T
        size(coupling, 1) == size(coupling, 2) || throw(DimensionMismatch("input must be square matrix."))
        new{T}(size(coupling, 1), coupling)
    end
end

"""
    load_coupling(filename::String) -> SpinAnnealingProblem

Load the data file into symmtric coupling matrix.
"""
function load_coupling(filename::String)
    data = readdlm(filename)
    # is = @. Int(view(data, :, 1)) + 1  #* @. means broadcast for the following functions, is here used correctly?
    # js = @. Int(view(data, :, 2)) + 1
    is = Int.(view(data, :, 1)) .+ 1
    js = Int.(view(data, :, 2)) .+ 1
    weights = data[:,3]
    num_spin = max(maximum(is), maximum(js))
    J = similar(weights, num_spin, num_spin)
    for (i, j, weight) = zip(is, js, weights)
        J[i,j] = weight/2
        J[j,i] = weight/2
    end
    SpinAnnealingProblem(J)
end

load_coupling

In [2]:
@testset "loading" begin
    sap = load_coupling("data/example.txt")
    @test size(sap.coupling) == (300, 300)
end

Test Summary: | Pass  Total
loading       |    1      1


Test.DefaultTestSet("loading", Any[], 1, false)

In [3]:
abstract type AnnealingConfig end

struct SpinConfig{Ts, Tf} <: AnnealingConfig
    config::Vector{Ts}
    field::Vector{Tf}
    SpinConfig(config::Vector{Ts}, field::Vector{Tf}) where {Ts, Tf} = new{Ts, Tf}(config, field)
end

"""
    random_config(prblm::AnnealingProblem) -> SpinConfig

Random spin configuration.
"""
function random_config end

function random_config(prblm::SpinAnnealingProblem)
    config = rand([-1,1], prblm.num_spin)
    SpinConfig(config, prblm.coupling*config)
end

random_config (generic function with 1 method)

In [4]:
@testset "random config" begin
    sap = load_coupling("data/example.txt")
    initial_config = random_config(sap)
    @test initial_config.config |> length == 300
    @test eltype(initial_config.config) == Int
end

Test Summary: | Pass  Total
random config |    2      2


Test.DefaultTestSet("random config", Any[], 2, false)

# Main Program for Annealing

In [5]:
"""
    anneal_singlerun!(config::AnnealingConfig, prblm, tempscales::Vector{Float64}, num_update_each_temp::Int)

Perform Simulated Annealing using Metropolis updates for the single run.

    * configuration that can be updated.
    * prblm: problem with `get_cost`, `flip!` and `random_config` interfaces.
    * tempscales: temperature scales, which should be a decreasing array.
    * num_update_each_temp: the number of update in each temprature scale.

Returns (minimum cost, optimal configuration).
"""
function anneal_singlerun!(config, prblm, tempscales::Vector{Float64}, num_update_each_temp::Int)
    cost = get_cost(config, prblm)
    
    opt_config = config
    opt_cost = cost
    for beta = 1 ./ tempscales
        @simd for m = 1:num_update_each_temp
            proposal, ΔE = propose(config, prblm)
            if exp(-beta*ΔE) > rand()  #accept
                flip!(config, proposal, prblm)
                cost += ΔE
                if cost < opt_cost
                    opt_cost = cost
                    opt_config = config
                end
            end
        end
    end
    opt_cost, opt_config
end
 
"""
    anneal(nrun::Int, prblm, tempscales::Vector{Float64}, num_update_each_temp::Int)

Perform Simulated Annealing with multiple runs.
"""
function anneal(nrun::Int, prblm, tempscales::Vector{Float64}, num_update_each_temp::Int)
    local opt_cost, opt_config
    for r = 1:nrun
        initial_config = random_config(prblm)
        cost, config = anneal_singlerun!(initial_config, prblm, tempscales, num_update_each_temp)
        if r == 1 || cost < opt_cost
            opt_cost = cost
            opt_config = config
        end
        println("$r-th run, cost = $cost")
    end
    opt_cost, opt_config
end

anneal

# Annealing Problem Interfaces

In [6]:
"""
    get_cost(config::AnnealingConfig, ap::AnnealingProblem) -> Real

Get the cost of specific configuration.
"""  # this kind of docstring is confusing
get_cost(config::SpinConfig, sap::SpinAnnealingProblem) = sum(config.config'*sap.coupling*config.config)

"""
    propose(config::AnnealingConfig, ap::AnnealingProblem) -> (Proposal, Real)

Propose a change, as well as the energy change.
"""
@inline function propose(config::SpinConfig, ::SpinAnnealingProblem)  # ommit the name of argument, since not used.
    ispin = rand(1:length(config.config))
    @inbounds ΔE = -config.field[ispin] * config.config[ispin] * 4 # 2 for spin change, 2 for mutual energy.
    ispin, ΔE
end

"""
    flip!(config::AnnealingConfig, ispin::Proposal, ap::AnnealingProblem) -> SpinConfig

Apply the change to the configuration.
"""
@inline function flip!(config::SpinConfig, ispin::Int, sap::SpinAnnealingProblem)
    @inbounds config.config[ispin] = -config.config[ispin]
    # update field
    @simd for i=1:sap.num_spin
        @inbounds config.field[i] += 2 * config.config[ispin] * sap.coupling[i,ispin]
    end
    config
    #* config.field .+= 2 .* config.config[ispin] .* sap.coupling[:,ispin]
end

flip!

### **Challege!**
Make your program correct and type is stable!

In [13]:
using Random
Random.seed!(2)
const tempscales = 10 .- (1:64 .- 1) .* 0.15 |> collect
const sap = load_coupling("data/example.txt")

SpinAnnealingProblem{Float64}(300, [0.0 0.5 … -0.5 -0.5; 0.5 0.0 … 0.5 -0.5; … ; -0.5 0.5 … 0.0 0.5; -0.5 -0.5 … 0.5 0.0])

In [43]:
@testset "anneal" begin
    opt_cost, opt_config = anneal(30, sap, tempscales, 4000)
    @test anneal(30, sap, tempscales, 4000)[1] == -3858
    anneal(30, sap, tempscales, 4000)
    res = median(@benchmark anneal(30, $sap, $tempscales, 4000))
    @test res.time/1e9 < 1
    @test res.allocs < 500
end

Test Summary: | Pass  Total
anneal        |    3      3


Test.DefaultTestSet("anneal", Any[], 3, false)

In [36]:
@benchmark anneal(30, $sap, $tempscales, 4000)

BenchmarkTools.TrialEstimate: 
  time:             678.755 ms
  gctime:           0.000 ns (0.00%)
  memory:           398.00 KiB
  allocs:           271

# Tips for optimization: Find the bottleneck of your program

In [36]:
using Profile
Profile.clear()
@profile anneal(100, sap, tempscales, 4000)
Profile.print()

2193 ./task.jl:259; (::getfield(IJulia, Symbol("##12#1...
 2193 ...k5o7j/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  2193 ./essentials.jl:685; invokelatest
   2193 ./essentials.jl:686; #invokelatest#1
    2193 ...c/execute_request.jl:165; execute_request(::ZMQ.Socket, :...
     2193 .../SoftGlobalScope.jl:212; softscope_include_string(::Modu...
      2193 ./boot.jl:319; eval
       2193 ./In[36]:3; top-level scope
        2193 ...file/src/Profile.jl:25; macro expansion
         30   ./In[23]:42; anneal(::Int64, ::SpinAnneali...
          30 ./In[21]:18; random_config(::SpinAnnealingP...
           30 ...ebra/src/matmul.jl:64; *
            30 ...bra/src/matmul.jl:357; gemv!(::Array{Float64,1}, ::...
             30 ...gebra/src/blas.jl:575; gemv!(::Char, ::Float64, ::...
         2163 ./In[23]:43; anneal(::Int64, ::SpinAnneali...
          13   ./In[23]:14; anneal_singlerun!(::SpinConf...
           13 ./In[24]:6; get_cost(::SpinConfig{Int64,Fl...
            13 ./operators.jl:502; *

# Calling a Fortran program
* https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/index.html
* https://craftofcoding.wordpress.com/2017/02/26/calling-fortran-from-julia-i/
* https://craftofcoding.wordpress.com/2017/03/01/calling-fortran-from-julia-ii/

In [68]:
;cd programs

/home/leo/jcode/JuliaChallenge/programs


In [69]:
;gfortran -shared -fPIC problem.f90 fsa.f90 -o fsa.so

In [70]:
;nm fsa.so

00000000000019e3 T anneal_
0000000000002017 T anneal_singlerun_
0000000000203128 B __bss_start
0000000000203140 b completed.7696
                 w __cxa_finalize@@GLIBC_2.2.5
00000000000011b0 t deregister_tm_clones
0000000000001240 t __do_global_dtors_aux
0000000000202de8 t __do_global_dtors_aux_fini_array_entry
0000000000203120 d __dso_handle
0000000000202df0 d _DYNAMIC
0000000000203128 D _edata
000000000025afc0 B _end
                 U expf@@GLIBC_2.27
00000000000022a4 T _fini
0000000000001280 t frame_dummy
0000000000202de0 t __frame_dummy_init_array_entry
0000000000002758 r __FRAME_END__
                 U free@@GLIBC_2.2.5
                 U _gfortran_arandom_r4@@GFORTRAN_7
                 U _gfortran_matmul_r4@@GFORTRAN_7
                 U _gfortran_os_error@@GFORTRAN_7
                 U _gfortran_random_r4@@GFORTRAN_7
                 U _gfortran_random_seed_i4@@GFORTRAN_7
                 U _gfortran_runtime_error_at@@GFORTRAN_7
                 U _gfortran_runtime_error@@G

In [ ]:
@benchmark ccall((:test_, "fsa.so"), Int32, ())

# What if I can not live without Python?
We can use [PyCall](https://github.com/JuliaPy/PyCall.jl) to call python programs!

### **Challenge!**
1. use Python package [viznet](https://github.com/GiggleLiu/viznet) and [matplotlib](https://matplotlib.org/) for visualization
2. benchmark pure python version of simulated annealing, show the time

In [1]:
# pip install viznet
using PyCall

@pyimport viznet
@pyimport matplotlib.pyplot as plt
brush = viznet.NodeBrush("nn.input")
brush >> (0, 0)
plt.axis([-1, 1, -1, 1])
plt.axis("equal")
plt.axis("off")
plt.show()

In [3]:
# now please import `test_codec` function in file `testsa.py`
pushfirst!(PyVector(pyimport("sys")["path"]), "programs")  # add current folder into path
@pyimport testsa
@benchmark testsa.test_codec()

LoadError: UndefVarError: @benchmark not defined